### Import Required Packages

In [1]:
%load_ext cython

In [2]:
import os
import sys
import git

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

### Attempt to Use Actual GCC Compiler

In [3]:
os.environ["CC"] = "gcc-9"

### Look into the Built-In Random Number Generator
* rand() gives a random integer `[0, RAND_MAX]`
* RAND_MAX is typically the max 32-bit integer value or `2,147,483,647`

In [4]:
%%cython --annotate

import cython
from libc.stdlib cimport rand, RAND_MAX
print("RAND_MAX:", RAND_MAX)

@cython.cdivision(True)
cpdef int random_item(int n_items):
    """sample a random item"""
    
    return rand() % n_items


RAND_MAX: 2147483647


In [5]:
# %%time
random_items = pd.Series([random_item(20) for _ in range(int(1e6))])
random_items.value_counts().sort_index()

0     49646
1     50376
2     50223
3     50182
4     49989
5     49891
6     49818
7     49844
8     49881
9     49926
10    50193
11    50295
12    49923
13    50081
14    49870
15    50329
16    49681
17    50055
18    49635
19    50162
dtype: int64

### Write Simple Utilities for Vector Addition/Subtraction

In [6]:
%%cython --annotate

import cython

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef void add_vv(float[::1] x, float[::1] y, float[::1] z):
    """add two vectors and put the result in a third"""
    
    cdef int i
    cdef int n = x.shape[0]
    for i in range(n):
        z[i] = x[i] + y[i]
        
@cython.wraparound(False)
@cython.boundscheck(False)
cpdef void sub_vv(float[::1] x, float[::1] y, float[::1] z):
    """subtract two vectors and put the result in a third"""
    
    cdef int i
    cdef int n = x.shape[0]
    for i in range(n):
        z[i] = x[i] - y[i]

In [23]:
x = np.random.random(50).astype(np.float32)
y = np.random.random(50).astype(np.float32)
z = np.zeros(50).astype(np.float32)

In [24]:
# %%timeit
x + y

array([1.1020935 , 1.2265073 , 1.6924734 , 1.4771678 , 0.7037684 ,
       1.0969839 , 1.0181234 , 0.5145763 , 1.139129  , 0.8576644 ,
       0.92918986, 0.47945997, 1.0375364 , 1.3454888 , 1.0763215 ,
       1.3193865 , 0.6892847 , 1.0588434 , 1.2501311 , 0.91633725,
       0.5701259 , 0.42322555, 1.2081959 , 1.0368273 , 1.0318793 ,
       0.14949186, 1.115659  , 0.81489503, 1.4250431 , 1.0372447 ,
       0.58350736, 0.7305732 , 1.1670665 , 0.13444921, 1.1688435 ,
       0.66342556, 0.7200762 , 1.2339566 , 0.7054565 , 0.90157634,
       1.1513093 , 1.5944263 , 1.7369201 , 0.4269787 , 1.4545605 ,
       0.76006675, 0.4343109 , 0.85099703, 0.41501987, 1.9236019 ],
      dtype=float32)

In [25]:
# %%timeit
add_vv(x, y, z)
np.asarray(z)

array([1.1020935 , 1.2265073 , 1.6924734 , 1.4771678 , 0.7037684 ,
       1.0969839 , 1.0181234 , 0.5145763 , 1.139129  , 0.8576644 ,
       0.92918986, 0.47945997, 1.0375364 , 1.3454888 , 1.0763215 ,
       1.3193865 , 0.6892847 , 1.0588434 , 1.2501311 , 0.91633725,
       0.5701259 , 0.42322555, 1.2081959 , 1.0368273 , 1.0318793 ,
       0.14949186, 1.115659  , 0.81489503, 1.4250431 , 1.0372447 ,
       0.58350736, 0.7305732 , 1.1670665 , 0.13444921, 1.1688435 ,
       0.66342556, 0.7200762 , 1.2339566 , 0.7054565 , 0.90157634,
       1.1513093 , 1.5944263 , 1.7369201 , 0.4269787 , 1.4545605 ,
       0.76006675, 0.4343109 , 0.85099703, 0.41501987, 1.9236019 ],
      dtype=float32)

In [26]:
x - y

array([ 0.5284644 ,  0.10067201, -0.06611842, -0.1488151 , -0.40776494,
        0.25595593,  0.12468943, -0.49301386,  0.2997059 , -0.37345976,
       -0.6957943 , -0.2715439 ,  0.6124034 ,  0.08320373, -0.29188105,
       -0.07384378,  0.00965557,  0.6485585 , -0.33711725,  0.16740093,
       -0.33551186,  0.2878637 ,  0.28516334,  0.9046644 , -0.81842166,
        0.05214965,  0.27715972, -0.08842486, -0.43653816,  0.9303699 ,
        0.43381995,  0.21389958,  0.4185945 ,  0.04499564, -0.07613581,
        0.15844262, -0.49385306, -0.6934849 , -0.6112206 , -0.6484193 ,
       -0.10898221, -0.3820306 ,  0.01120895,  0.32114536,  0.29752737,
       -0.15444556, -0.42470425,  0.3564517 , -0.16258824,  0.0346117 ],
      dtype=float32)

In [27]:
# %%timeit
sub_vv(x, y, z)
np.asarray(z)

array([ 0.5284644 ,  0.10067201, -0.06611842, -0.1488151 , -0.40776494,
        0.25595593,  0.12468943, -0.49301386,  0.2997059 , -0.37345976,
       -0.6957943 , -0.2715439 ,  0.6124034 ,  0.08320373, -0.29188105,
       -0.07384378,  0.00965557,  0.6485585 , -0.33711725,  0.16740093,
       -0.33551186,  0.2878637 ,  0.28516334,  0.9046644 , -0.81842166,
        0.05214965,  0.27715972, -0.08842486, -0.43653816,  0.9303699 ,
        0.43381995,  0.21389958,  0.4185945 ,  0.04499564, -0.07613581,
        0.15844262, -0.49385306, -0.6934849 , -0.6112206 , -0.6484193 ,
       -0.10898221, -0.3820306 ,  0.01120895,  0.32114536,  0.29752737,
       -0.15444556, -0.42470425,  0.3564517 , -0.16258824,  0.0346117 ],
      dtype=float32)

### Try to Get BLAS Wrappers Working

#### Wrap Vector-Vector and Matrix-Vector Dot Product Functions

In [131]:
%%cython

from scipy.linalg.cython_blas cimport sdot, sgemv

cpdef float dot_vv(float[::1] x, float[::1] y):
    """compute a vector-vector dot product"""
    
    # set the vector length and stride
    cdef int n = x.shape[0], incx = 1, incy = 1
    
    # call the underlying BLAS routine and return the scalar result
    return sdot(&n, &x[0], &incx, &y[0], &incy)


cpdef void dot_mv(float[::1, :] A, float[::1] x, float[::1] y):
    """compute a matrix-vector dot product"""
        
    # set the matrix dimensions
    cdef int m = A.shape[0], n = A.shape[1]
    
    # set the alpha/beta scalars
    cdef float alpha = 1.0, beta = 0.0
    
    # set the x/y stride
    cdef int incx = 1, incy = 1
    
    # call the underlying BLAS routine and store result in [y]
    # NOTE: the matrix [A] must be F-CONTIGUOUS for this to work as written
    # NOTE: it would be much better to figure out how to get this to work C-CONTIGUOUS
    # NOTE: the result is placed into the vector [y] instead of being returned
    sgemv('N', &m, &n, &alpha, &A[0, 0], &m, &x[0], &incx, &beta, &y[0], &incy)




#### Confirm the Accuracy of the Results

##### vector-vector dot product

In [132]:
x = np.random.randn(100).astype(np.float32)
y = np.random.randn(100).astype(np.float32)

In [133]:
# %%timeit
np.dot(x, y)

8.497783

In [134]:
# %%timeit
dot_vv(x, y)

8.497782707214355

##### matrix-vector dot product

In [135]:
A = np.random.random(10000).reshape((1000, 10)).T.astype(np.float32)
x = np.random.random(1000).astype(np.float32)
y = np.zeros(10, dtype=np.float32)

print(A.shape)
print(x.shape)
print(y.shape)
A.flags

(10, 1000)
(1000,)
(10,)


  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [136]:
# %%timeit
np.dot(A, x)

array([249.4627 , 252.2492 , 250.55032, 247.5303 , 259.55026, 263.7343 ,
       258.11224, 259.49347, 247.08813, 251.36603], dtype=float32)

In [140]:
# %%timeit
dot_mv(A, x, y)
y

array([249.4627 , 252.2492 , 250.55032, 247.5303 , 259.55026, 263.7343 ,
       258.11224, 259.49347, 247.08813, 251.36603], dtype=float32)

### Try to Get Basic NOGIL Functionality Working
* implement a linear search function to see if a given item is in a user's items
* implement C arrays to hold both each user's number of items and each user's set of items
* all of this must be able to be used in a NOGIL block to parallelize the eventual training loop

In [91]:
%%cython --annotate

cimport cython
from libc.stdlib cimport malloc, free


cdef int lsearch(int item, int *items, int length) nogil:
    """linear search for a given item in a sorted array of items"""
    
    cdef int i
    for i in range(length):
        if item == items[i]:
            return True
    return False


def process(dict p_user_items):
    """create a C array and try to use it in a [nogil] block"""
    
    # declare all C variables
    cdef int user
    cdef int item
    cdef int n_users
    cdef int *n_items
    cdef int **c_user_items
    
    # count the total users and number of items for each user
    n_users = len(p_user_items)
    n_user_items = {user: len(items) for user, items in p_user_items.items()}
    
    # initialize the [n_items] and [c_user_items] C arrays
    n_items = <int*>malloc(n_users * sizeof(int))
    c_user_items = <int**>malloc(n_users * sizeof(int*))
    
    # fill the C arrays
    for user in range(n_users):
        
        # fill the item count array for each user
        n_items[user] = n_user_items[user]
        
        # allocate the dynamic item set array for each user
        c_user_items[user] = <int*>malloc(n_items[user] * sizeof(int))
        
        # fill the item set array for each user
        for item in range(n_items[user]):
            c_user_items[user][item] = p_user_items[user][item]
            
    # confirm C array values in a NOGIL block
    with nogil:
        for user in range(n_users):
            res = lsearch(9, c_user_items[user], n_items[user])
            for item in range(n_items[user]):
                c_user_items[user][item] += 10
                with gil:
                    print("user:", user, "item:", c_user_items[user][item], "number of items:", n_items[user], "found:", res)
                
    # free the dynamic item arrays for each user 
    for user in range(n_users):
        free(c_user_items[user])
        
    # free the item count and top-level user items arrays
    free(n_items)
    free(c_user_items)
        


In [84]:
repo_root = "/Users/ericlundquist/Repos/rankfm"
data_path = os.path.join(repo_root, "data/examples")
print("\n".join([repo_root, data_path]))

/Users/ericlundquist/Repos/rankfm
/Users/ericlundquist/Repos/rankfm/data/examples


In [85]:
interactions = pd.read_csv(os.path.join(data_path, 'ML_1M_RATINGS.csv'))
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
user_id      1000209 non-null int64
item_id      1000209 non-null int64
rating       1000209 non-null int64
timestamp    1000209 non-null object
dtypes: int64(3), object(1)
memory usage: 30.5+ MB


In [86]:
interactions['user_id'] = interactions['user_id'] - 1
interactions['item_id'] = interactions['item_id'] - 1
interactions = interactions.sort_values(['user_id', 'item_id']).groupby('user_id')['item_id'].apply(np.array, dtype=np.int32).to_dict()
print(len(interactions))
interactions[0]

6040


array([   0,   47,  149,  259,  526,  530,  587,  593,  594,  607,  660,
        719,  744,  782,  913,  918,  937, 1021, 1027, 1028, 1034, 1096,
       1192, 1196, 1206, 1245, 1269, 1286, 1544, 1565, 1720, 1835, 1906,
       1960, 1961, 2017, 2027, 2293, 2320, 2339, 2354, 2397, 2686, 2691,
       2761, 2790, 2796, 2803, 2917, 3104, 3113, 3185, 3407], dtype=int32)

In [92]:
p_user_items = {
    0: np.array([9, 6, 3], dtype=np.int32),
    1: np.array([2, 4, 6, 8, 10], dtype=np.int32)
}
p_user_items

{0: array([9, 6, 3], dtype=int32), 1: array([ 2,  4,  6,  8, 10], dtype=int32)}

In [94]:
%%time
process(p_user_items)

user: 0 item: 19 number of items: 3 found: 1
user: 0 item: 16 number of items: 3 found: 1
user: 0 item: 13 number of items: 3 found: 1
user: 1 item: 12 number of items: 5 found: 0
user: 1 item: 14 number of items: 5 found: 0
user: 1 item: 16 number of items: 5 found: 0
user: 1 item: 18 number of items: 5 found: 0
user: 1 item: 20 number of items: 5 found: 0
CPU times: user 1.46 ms, sys: 1.73 ms, total: 3.19 ms
Wall time: 1.81 ms


# **START EXAMPLE CODE FROM ONLINE RESOURCES**

### Chapter 1 - Cython Essentials

#### Sample Python Program

In [3]:
def p_fib(n):
    a, b = 0.0, 1.0
    for i in range(n):
        a, b = a + b, a
    return a

#### Equivalent Cython Program

In [4]:
%%cython --annotate

def c_fib(int n):
    cdef int i
    cdef double a=0.0, b=1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [5]:
%%timeit
p_fib(1000)

41.5 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
%%timeit
c_fib(1000)

1.22 µs ± 29.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Chapter 9 - Memoryviews

In [18]:
%%cython

def summer_1(float[:] mv):
    """iterate over a data buffer and compute the sum"""
    
    cdef:
        double d
        ss = 0.0
    for d in mv:
        ss += d
    return ss

#### Faster Version with Direct Access and No Python Overhead
* turn off `boundscheck` and `wraparound`
* use a typed iterator (i) and range(N) for looping
* notice zero python interaction in the function body

In [29]:
%%cython --annotate

from cython cimport boundscheck, wraparound

@boundscheck(False)
@wraparound(False)
def summer_2(float[:] mv):
    """loop with a typed range and iterator to index directly into the buffer with no Python overhead"""
    
    cdef:
        int i, N
        float ss = 0.0
        
    # easily calculate number of elements
    N = mv.shape[0]
    
    # loop over elements and index directly
    for i in range(N):
        ss += mv[i]
        
    return ss

In [30]:
to_sum = np.ones(1000000, dtype=np.float32)

In [31]:
%%timeit
summer_1(to_sum)

150 ms ± 873 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%%timeit
summer_2(to_sum)

1.11 ms ± 8.41 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [36]:
%%cython --annotate

from cython cimport boundscheck, wraparound

@boundscheck(False)
@wraparound(False)
def mv_sum(int[:, ::1] mv):
    """sum the elements of a 2D array"""
    
    cdef int N, M, i, j
    cdef long res = 0
    
    N = mv.shape[0]
    M = mv.shape[1]
    
    for i in range(N):
        for j in range(M):
            res += mv[i,j]
            
    return res

In [38]:
to_sum = np.ones((100, 100), dtype=np.int32)

In [39]:
mv_sum(to_sum)

10000

# **START EXAMPLE CODE**

### Bigger Example Using Typed MemoryViews

In [43]:
%%cython

from array import array
from math import sqrt

from cython cimport cdivision, boundscheck, wraparound
from cpython.array cimport array

@cdivision(True)
cdef inline double A(int i, int j):
    """pulls an element from a predictable infinite matrix"""
    return 1.0 / (((i + j) * (i + j + 1) >> 1) + i + 1)

@boundscheck(False)
@wraparound(False)
cdef void A_times_u(double[::1] u, double[::1] v):
    """matrix-vector dot product - store result in vector [v]"""
    

    cdef int i, j
    cdef u_len = u.shape[0]
    cdef double partial_sum

    for i in range(u_len):
        partial_sum = 0
        for j in range(u_len):
            partial_sum += A(i, j) * u[j]
        v[i] = partial_sum


@boundscheck(False)
@wraparound(False)
cdef void At_times_u(double[::1] u, double[::1] v):
    """matrix-vector dot product on A-transpose"""
    
    cdef:
        int i, j
        u_len = u.shape[0]
        double partial_sum

    for i in range(u_len):
        partial_sum = 0
        for j in range(u_len):
            partial_sum += A(j, i) * u[j]
        v[i] = partial_sum


def B_times_u(u, out, tmp):
    """swapping helper function"""
    
    A_times_u(u, tmp)
    At_times_u(tmp, out)


def spectral_norm(n):
    """compute the final spectral norm"""
    
    u = array("d", [1.0] * n)
    v = array("d", [0.0] * n)
    tmp = array("d", [0.0] * n)

    for _ in range(10):
        B_times_u(u, v, tmp)
        B_times_u(v, u, tmp)

    vBv = vv = 0

    for ue, ve in zip(u, v):
        vBv += ue * ve
        vv  += ve * ve

    return sqrt(vBv / vv)

### Another Example Using Typed MemoryViews

In [51]:
%%cython --annotate

from cython cimport cdivision, boundscheck, wraparound
import numpy as np
DTYPE = np.intc


cdef int clip(int a, int min_value, int max_value):
    return min(max(a, min_value), max_value)


@boundscheck(False)
@wraparound(False)
def compute_seq(int[:, :] array_1, int[:, :] array_2, int a, int b, int c):

    # declare loop ranges as C-int types
    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]
    assert tuple(array_1.shape) == tuple(array_2.shape)

    # create a numpy array to hold results and a memory view to access it
    result = np.zeros((x_max, y_max), dtype=DTYPE)
    cdef int[:, :] result_view = result

    # declare temporary loop variables and loop iterators as C types
    cdef int tmp
    cdef Py_ssize_t x, y

    # fast loops in C filling the values of the result array
    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    # return the result which will be a numpy array
    return result

### Parallel Version

In [68]:
os.environ['gcc'] = 'gcc-9'

In [69]:
%%cython --compile-args=/openmp --link-args=/openmp --force
cimport cython
cimport openmp

import cython.parallel as cp
from cython.parallel import parallel, prange

import numpy as np
cimport cython

ctypedef fused my_type:
    int
    double
    long long


# We declare our plain c function nogil
cdef my_type clip(my_type a, my_type min_value, my_type max_value) nogil:
    return min(max(a, min_value), max_value)


@cython.boundscheck(False)
@cython.wraparound(False)
def compute_par(my_type[:, ::1] array_1, my_type[:, ::1] array_2, my_type a, my_type b, my_type c):

    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]

    assert tuple(array_1.shape) == tuple(array_2.shape)

    if my_type is int:
        dtype = np.intc
    elif my_type is double:
        dtype = np.double
    elif my_type is cython.longlong:
        dtype = np.longlong

    result = np.zeros((x_max, y_max), dtype=dtype)
    cdef my_type[:, ::1] result_view = result

    cdef my_type tmp
    cdef Py_ssize_t x, y

    # We use prange here.
    for x in prange(x_max, nogil=True):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    return result

In [60]:
import numpy as np
array_1 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)
array_2 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)
a = 4
b = 3
c = 9

In [61]:
%timeit compute_seq(array_1, array_2, a, b, c)

16.9 ms ± 261 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [70]:
%timeit compute_par(array_1, array_2, a, b, c)

NameError: name 'compute_par' is not defined

### Parallel Programming with PRANGE()

In [ ]:
%%cython

# distutils: extra_compile_args = -fopenmp
# distutils: extra_link_args = -fopenmp

from cython cimport boundscheck, wraparound
from cython.parallel cimport prange

import numpy as np

cdef inline double norm2(double complex z) nogil:
    return z.real * z.real + z.imag * z.imag


cdef int escape(double complex z, double complex c, double z_max, int n_max) nogil:

    cdef:
        int i = 0
        double z_max2 = z_max * z_max

    while norm2(z) < z_max2 and i < n_max:
        z = z * z + c
        i += 1

    return i


@boundscheck(False)
@wraparound(False)
def calc_julia(int resolution, double complex c, double bound=1.5, double z_max=4.0, int n_max=1000):

    cdef:
        double step = 2.0 * bound / resolution
        int i, j
        double complex z
        double real, imag
        int[:, ::1] counts

    counts = np.zeros((resolution+1, resolution+1), dtype=np.int32)

    for i in prange(resolution + 1, nogil=True, schedule='static', chunksize=1):
        real = -bound + i * step
        for j in range(resolution + 1):
            imag = -bound + j * step
            z = real + imag * 1j
            counts[i,j] = escape(z, c, z_max, n_max)

    return np.asarray(counts)

@boundscheck(False)
@wraparound(False)
def julia_fraction(int[:,::1] counts, int maxval=1000):
    cdef:
        int total = 0
        int i, j, N, M
        
    N = counts.shape[0] 
    M = counts.shape[1]

    for i in prange(N, nogil=True):
        for j in range(M):
            if counts[i,j] == maxval:
                total += 1
    return total / float(counts.size)